   #              MNIST Classification with Tensorflow RNNs

### Loading TensorBoard

In [1]:
%load_ext tensorboard.notebook

In [2]:
import tensorflow as tf

C:\Users\kmkar\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kmkar\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kmkar\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kmkar\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

### Getting MNIST data

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data" , one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### Hyperparameters
#### Log data is to stored in logs/RNN_with_summaries

In [4]:
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

LOG_DIR = 'logs/RNN_with_summaries'

### Defining Placeholders

In [5]:
_inputs = tf.placeholder(tf.float32 , shape = [None , time_steps , element_size], name = 'inputs')
y = tf.placeholder(tf.float32 , shape = [None , num_classes] , name = 'labels')

In [6]:
batch_x , batch_y = mnist.train.next_batch(batch_size)
batch_x = batch_x.reshape((batch_size , time_steps , element_size))

In [7]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean' , mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev' , stddev)
        tf.summary.scalar('max' , tf.reduce_max(var))
        tf.summary.scalar('min' , tf.reduce_min(var))
        tf.summary.histogram('histogram' , var)        

### Defining histograms in TensorBoard for Weights and Biases of the RNN

In [8]:
with tf.name_scope('rnn_weights'):
    with tf.name_scope('w_x'):
        Wx = tf.Variable(tf.zeros([element_size , hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope('W_h'):
        Wh = tf.Variable(tf.zeros([hidden_layer_size , hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope('Bias'):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)        

Instructions for updating:
Colocations handled automatically by placer.


### Defining the RNN Model

In [9]:
def rnn_step(previous_hidden_state , x):
    current_hidden_state = tf.tanh(tf.matmul(previous_hidden_state , Wh) + tf.matmul(x , Wx) + b_rnn)
    return current_hidden_state

In [10]:
def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state , Wl)+bl

### Defining the RNN process 
##### tf.scan() iteratively runs through rnn_step

In [11]:
processed_input = tf.transpose(_inputs , perm = [1,0,2])
initial_hidden = tf.zeros([batch_size , hidden_layer_size])
all_hidden_states = tf.scan(rnn_step , processed_input , initializer = initial_hidden , name = 'states')

In [12]:
with tf.name_scope("linear_layer_weights") as scope :
    with tf.name_scope("W_linear"):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size , num_classes] ,mean = 0 , stddev = 0.1))
        variable_summaries(Wl)
        
    with tf.name_scope("Bias_linear"):
        bl = tf.Variable(tf.truncated_normal([num_classes] ,mean = 0, stddev = 0.1))
        variable_summaries(bl)

In [13]:
with tf.name_scope("linear_layer_weights") as scope :
    all_outputs = tf.map_fn(get_linear_layer , all_hidden_states)
    output = all_outputs[-1]
    tf.summary.histogram('outputs' , output)

### Writing Accuracy and Loss to TensorBoard
### Defining the Optimizer = RMSProp

In [14]:
with tf.name_scope("cross_entropy") :
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = output , labels = y))
    tf.summary.scalar( 'cross_entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.RMSPropOptimizer(0.001 , 0.9).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1) , tf.argmax(output , 1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction , tf.float32)))*100
    tf.summary.scalar( 'accuracy', accuracy)

merged = tf.summary.merge_all()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Reshaping Test Data

In [15]:
test_data = mnist.test.images[:batch_size].reshape((-1 , time_steps , element_size))
test_label = mnist.test.labels[:batch_size]

### Running Session

In [16]:
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train' , graph = tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test' , graph = tf.get_default_graph())
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        batch_x , batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size , time_steps , element_size))
        summary,_ = sess.run([merged , train_step] , feed_dict = {_inputs:batch_x , y:batch_y})
        train_writer.add_summary(summary , i)
        if i%1000 ==0 :
            acc,loss, = sess.run([accuracy , cross_entropy] , feed_dict = {_inputs:batch_x , y:batch_y})
            print("Iter " + str(i) + ", Minibatch Loss = " + "{:.6f}".format(loss) + ", Training Accuracy = " + "{:.5f}".format(acc))
        
        if i%10:
            summary,acc = sess.run([merged , accuracy] , feed_dict = {_inputs:test_data , y:test_label})
            train_writer.add_summary(summary , i)
    
    test_acc = sess.run(accuracy , feed_dict = {_inputs:test_data , y:test_label})
    print("Test Accuracy:" , test_acc)

Iter 0, Minibatch Loss = 2.297724, Training Accuracy = 11.71875
Iter 1000, Minibatch Loss = 0.646977, Training Accuracy = 75.78125
Iter 2000, Minibatch Loss = 0.214019, Training Accuracy = 93.75000
Iter 3000, Minibatch Loss = 0.280971, Training Accuracy = 91.40625
Iter 4000, Minibatch Loss = 0.086724, Training Accuracy = 97.65625
Iter 5000, Minibatch Loss = 0.050195, Training Accuracy = 98.43750
Iter 6000, Minibatch Loss = 0.164306, Training Accuracy = 96.09375
Iter 7000, Minibatch Loss = 0.024312, Training Accuracy = 100.00000
Iter 8000, Minibatch Loss = 0.051649, Training Accuracy = 98.43750
Iter 9000, Minibatch Loss = 0.080767, Training Accuracy = 98.43750
Test Accuracy: 95.3125
